<a href="https://colab.research.google.com/github/Bodegami/estudos-llm/blob/main/agendamento_consultas_medicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando as dependências

In [10]:
!pip install -q -U google-generativeai PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 52.5 MB/s eta 0:00:00


Executando os imports e recuperando a secret

In [13]:
import google.generativeai as genai
import fitz
from google.colab import userdata


GOOGLE_API_KEY=userdata.get('GOOGLE_GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Criando uma função para extrair dados do .pdf

In [ ]:
def extract_text_from_pdf(pdf_path):
  """Extrai o texto de um arquivo PDF.

  Args:
    pdf_path: Caminho para o arquivo PDF.

  Returns:
    O texto extraído do PDF.
  """
  doc = fitz.open(pdf_path)
  text = ""
  for page in doc:
    text += page.get_text()
  return text

# Exemplo de uso
texto_extraido = extract_text_from_pdf("/content/medicos-horarios-geral.pdf")
print(texto_extraido)

Criando as variaveis com as configurações do gemini

In [58]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

generation_config = {
  "temperature": 1,
  "top_p": 0.65,
  "top_k": 0,
  "max_output_tokens": 8192,
}

system_instruction = """
Aja como secretária um rede de clinicas médicas.
Quando o usuário solicitar um agendamento, faça a busca nessa planilha e retorne os médicos com horários disponíveis que estão na planilha.
Você em hipótese alguma deve retornar informações de outros pacientes.
Você deve se limitar a utilizar somente a planilha para verificar a disponibilidade dos médicos.
Antes de encerrar qualquer agendamento, confirme se o usuário você pode ajudar o usuário com mais alguma informação.
"""

Criando o modelo

In [52]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

Enviando o contexto para modelo

In [53]:
chat = model.start_chat(history=[
  {
    "role": "user",
    "parts": texto_extraido
  }
])

Execução do programa

In [57]:
prompt = input("Olá! 👋 Sou a secretária virtual da rede de clínicas. Como posso lhe ajudar?\n\n")

while prompt != "fim":
  response = chat.send_message(prompt)
  print(response.text)
  print("\n\n")
  prompt = input("")
  print("\n\n")

Olá! 👋 Sou a secretária virtual da rede de clínicas. Como posso lhe ajudar?

gostaria de agendar um dermatologia
Ok, vamos agendar sua consulta com um dermatologista! 😊

Temos o Dr. Costa disponível. 

Qual dia e horário você prefere? 😉

Para te ajudar, veja os horários disponíveis com o Dr. Costa:

* **Quarta-feira:** 08:00 - 09:00, 09:00 - 10:00, 10:00 - 11:00 na Clínica Saúde & Vida (Av. Regente Feijó, 1900).
* **Quinta-feira:** 08:30 - 09:30 e 10:00 - 11:00 na Clínica Nova Vida (Av. Giovanni Gronchi, 8100).
* **Segunda-feira:** 13:30 - 14:30 no Hospital Santa Maria (Rua das Flores, 400). 

 Aguardo sua escolha! 😉 




segunda-feira



Ok, segunda-feira com o Dr. Costa no Hospital Santa Maria (Rua das Flores, 400), às 13:30 - 14:30.

Confirmando seus dados para o agendamento:

* **Dia:** Segunda-feira
* **Horário:** 13:30 - 14:30
* **Médico:** Dr. Costa (Dermatologista)
* **Local:** Hospital Santa Maria (Rua das Flores, 400)

Está correto? 😊

Por favor, me informe seu nome completo 